In [ ]:
import boto3

client = boto3.client('ec2')

In [ ]:
my_session = boto3.session.Session()
my_region = my_session.region_name
print my_region

In [ ]:
# AMIs are unique per region. Here we will look up our AMI based on our region
amis = {}
amis['us-east-1'] = 'ami-0b33d91d'
amis['us-east-2'] = 'ami-446f3521'
amis['us-west-1'] = 'ami-9fadf8ff'
amis['us-west-2'] = 'ami-7abc111a'
amis['eu-west-1'] = 'ami-a1491ad2'
amis['ca-central-1'] = 'ami-ebed508f'

ami_id = amis[my_region]
print ami_id

In [ ]:
#Create security group
response = client.create_security_group(
    GroupName='web_sg',
    Description='Use for launching public web service'
)

webSgID = response['GroupId']
print 'created security group ', webSgID

In [ ]:
# Allow ingress on port 80
response = client.authorize_security_group_ingress(
    GroupId=webSgID,
    IpProtocol='tcp',
    FromPort=80,
    ToPort=80,
    CidrIp='0.0.0.0/0'
)

print response

In [ ]:
# Allow ingress for ssh
response = client.authorize_security_group_ingress(
    GroupId=webSgID,
    IpProtocol='tcp',
    FromPort=22,
    ToPort=22,
    CidrIp='0.0.0.0/0'
)

print response

In [ ]:
import os

keyName = os.environ["KEYNAME"]
print keyName

In [ ]:
user_data = \
"""#!/bin/bash
yum update -y
yum install httpd -y
cd /var/www/html
echo "<html><h1>Hello cloud gurus</h1></html>" > /var/www/html/index.html
service httpd start"""

In [ ]:
# Launch instance - default VPC, default subnet, etc.
launch_response = client.run_instances(
    KeyName = keyName,
    DryRun=False,
    ImageId=ami_id,
    MinCount=1,
    MaxCount=1,
    InstanceType='t2.micro',
    UserData=user_data,
    SecurityGroups=[
        'web_sg'
    ],
)


instanceId = launch_response['Instances'][0]['InstanceId']

In [ ]:
# Wait for instance
print 'Waiting for launch of ', instanceId
waiter = client.get_waiter('instance_running')
waiter.wait(
    InstanceIds=[
        instanceId,
    ]
)
print 'Launch complete'

In [ ]:
response = client.describe_instances(
InstanceIds=[
        instanceId,
    ]
)

print response['Reservations'][0]['Instances'][0]['PublicDnsName']

In [ ]:
# Terminate the EC2 Instance
response = client.terminate_instances(
    InstanceIds=[
        instanceId,
    ]
)

print response

print 'Waiting for termination to complete...'
waiter = client.get_waiter('instance_terminated')
waiter.wait(
   InstanceIds=[
       instanceId,
   ]
)
print 'Terminated'

In [ ]:
# Clean up security group
response = client.delete_security_group(
    GroupId=webSgID
)

print response

In [ ]:
# Future notebooks - EBS volume - boot (3 types), additional volumes (all 5), delete on terminate vs
# do not delete on terminate.